In [1]:
from pennylane import numpy as np
import pennylane as qml
from matplotlib import pyplot as plt

from functions_vqe_th import vqe_hee, vqe_ng
import numpy as np
from joblib import Parallel, delayed

device = 'cpu'

## AntiF Heisenberg hamiltonian definition 

In [14]:
n_cells = (3, 3)
coupling = [1,1,1]

qubits = n_cells[0] * n_cells[1]

lattice = 'triangle'

H = qml.spin.heisenberg(lattice, n_cells, coupling, boundary_condition=True)

H

(
    X(6) @ X(8)
  + Y(6) @ Y(8)
  + Z(6) @ Z(8)
  + X(2) @ X(8)
  + Y(2) @ Y(8)
  + Z(2) @ Z(8)
  + X(6) @ X(7)
  + Y(6) @ Y(7)
  + Z(6) @ Z(7)
  + X(2) @ X(6)
  + Y(2) @ Y(6)
  + Z(2) @ Z(6)
  + X(0) @ X(6)
  + Y(0) @ Y(6)
  + Z(0) @ Z(6)
  + X(7) @ X(8)
  + Y(7) @ Y(8)
  + Z(7) @ Z(8)
  + X(0) @ X(7)
  + Y(0) @ Y(7)
  + Z(0) @ Z(7)
  + X(1) @ X(7)
  + Y(1) @ Y(7)
  + Z(1) @ Z(7)
  + X(1) @ X(8)
  + Y(1) @ Y(8)
  + Z(1) @ Z(8)
  + X(0) @ X(2)
  + Y(0) @ Y(2)
  + Z(0) @ Z(2)
  + X(2) @ X(5)
  + Y(2) @ Y(5)
  + Z(2) @ Z(5)
  + X(0) @ X(1)
  + Y(0) @ Y(1)
  + Z(0) @ Z(1)
  + X(0) @ X(5)
  + Y(0) @ Y(5)
  + Z(0) @ Z(5)
  + X(0) @ X(3)
  + Y(0) @ Y(3)
  + Z(0) @ Z(3)
  + X(1) @ X(2)
  + Y(1) @ Y(2)
  + Z(1) @ Z(2)
  + X(1) @ X(3)
  + Y(1) @ Y(3)
  + Z(1) @ Z(3)
  + X(1) @ X(4)
  + Y(1) @ Y(4)
  + Z(1) @ Z(4)
  + X(2) @ X(4)
  + Y(2) @ Y(4)
  + Z(2) @ Z(4)
  + X(3) @ X(5)
  + Y(3) @ Y(5)
  + Z(3) @ Z(5)
  + X(5) @ X(8)
  + Y(5) @ Y(8)
  + Z(5) @ Z(8)
  + X(3) @ X(4)
  + Y(3) @ Y(4)
  + Z(

## Distribution of results

In [3]:
max_iterations = 150

layers = np.arange(2, 17, 4)
n_layers = len(layers)

n_runs = 50

conv_tol = 1e-06

weights_lr = 0.1
phi_lr = 0.1

In [4]:
def run_single_simulation_1(i, l, j):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_hee(H, qubits, l, lr=weights_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)
    
    return i, j, (energy, grad_norm, grad_var, conv)

def run_single_simulation_2(i, l, j):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_ng(H, qubits, l, weights_lr=weights_lr, phi_lr=phi_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)
    
    return i, j, (energy, grad_norm, grad_var, conv)

### HEE

In [5]:
tasks = (delayed(run_single_simulation_1)(i, l, j)
         for i, l in enumerate(layers)
         for j in range(n_runs))
print('number of tasks:', n_layers * n_runs)
results = Parallel(n_jobs=-1, verbose=11)(tasks)

number of tasks: 200


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   18.6s
[Paralle

In [6]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_layers, n_runs, max_iterations+1))
grad_norms = np.zeros((n_layers, n_runs, max_iterations))
grad_variances = np.zeros((n_layers, n_runs, max_iterations))
convergences = np.zeros((n_layers, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]

In [7]:
prefix = "../local_data/log4/"
spec = "_cnot_1"

np.save(prefix+"energies"+spec+".npy", energies)
np.save(prefix+"grad_norms"+spec+".npy", grad_norms)
np.save(prefix+"grad_variances"+spec+".npy", grad_variances)
np.save(prefix+"convergences"+spec+".npy",convergences)

### NG

In [8]:
tasks = (delayed(run_single_simulation_2)(i, l, j)
         for i, l in enumerate(layers)
         for j in range(n_runs))
print('number of tasks:', n_layers * n_runs)
results = Parallel(n_jobs=-1, verbose=11)(tasks)

number of tasks: 200


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  

In [9]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_layers, n_runs, max_iterations+1))
grad_norms = np.zeros((n_layers, n_runs, max_iterations))
grad_variances = np.zeros((n_layers, n_runs, max_iterations))
convergences = np.zeros((n_layers, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]

In [10]:
prefix = "../local_data/log4/"
spec = "_cnot_2"

np.save(prefix+"energies"+spec+".npy", energies)
np.save(prefix+"grad_norms"+spec+".npy", grad_norms)
np.save(prefix+"grad_variances"+spec+".npy", grad_variances)
np.save(prefix+"convergences"+spec+".npy",convergences)